# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 22 2022 11:18AM,247055,19,ACG-2102491318,ACG North America LLC,Released
1,Sep 22 2022 11:15AM,247053,10,0086000700,ISDIN Corporation,Released
2,Sep 22 2022 11:15AM,247053,10,0086000706,ISDIN Corporation,Released
3,Sep 22 2022 11:15AM,247053,10,0086000707,ISDIN Corporation,Released
4,Sep 22 2022 11:15AM,247053,10,0086000714,ISDIN Corporation,Released
5,Sep 22 2022 11:15AM,247053,10,0086000758,ISDIN Corporation,Released
6,Sep 22 2022 11:15AM,247053,10,0086000764,ISDIN Corporation,Released
7,Sep 22 2022 11:15AM,247053,10,0086000761,ISDIN Corporation,Released
8,Sep 22 2022 11:15AM,247053,10,0086000774,ISDIN Corporation,Released
9,Sep 22 2022 11:15AM,247053,10,0086002606,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,247051,Released,1
35,247052,Released,7
36,247053,Released,11
37,247054,Released,1
38,247055,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247051,NaN,NaN,1.0
247052,NaN,NaN,7.0
247053,NaN,NaN,11.0
247054,NaN,NaN,1.0
247055,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246889,0.0,1.0,0.0
246890,0.0,0.0,1.0
246933,0.0,1.0,0.0
246951,5.0,1.0,1.0
246953,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246889,0,1,0
246890,0,0,1
246933,0,1,0
246951,5,1,1
246953,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246889,0,1,0
1,246890,0,0,1
2,246933,0,1,0
3,246951,5,1,1
4,246953,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246889,,1,
1,246890,,,1
2,246933,,1,
3,246951,5,1,1
4,246953,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 22 2022 11:18AM,247055,19,ACG North America LLC
1,Sep 22 2022 11:15AM,247053,10,ISDIN Corporation
12,Sep 22 2022 11:09AM,247054,19,"NAPP Technologies, LLC"
13,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc."
48,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation
55,Sep 22 2022 11:02AM,247051,10,ISDIN Corporation
56,Sep 22 2022 10:59AM,247050,10,"Methapharm, Inc."
57,Sep 22 2022 10:58AM,247049,10,"Methapharm, Inc."
58,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.
63,Sep 22 2022 10:56AM,247046,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 22 2022 11:18AM,247055,19,ACG North America LLC,,,1
1,Sep 22 2022 11:15AM,247053,10,ISDIN Corporation,,,11
2,Sep 22 2022 11:09AM,247054,19,"NAPP Technologies, LLC",,,1
3,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc.",,,35
4,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation,,,7
5,Sep 22 2022 11:02AM,247051,10,ISDIN Corporation,,,1
6,Sep 22 2022 10:59AM,247050,10,"Methapharm, Inc.",,,1
7,Sep 22 2022 10:58AM,247049,10,"Methapharm, Inc.",,,1
8,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.,,,5
9,Sep 22 2022 10:56AM,247046,10,"Methapharm, Inc.",,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 11:18AM,247055,19,ACG North America LLC,1,,
1,Sep 22 2022 11:15AM,247053,10,ISDIN Corporation,11,,
2,Sep 22 2022 11:09AM,247054,19,"NAPP Technologies, LLC",1,,
3,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc.",35,,
4,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation,7,,
5,Sep 22 2022 11:02AM,247051,10,ISDIN Corporation,1,,
6,Sep 22 2022 10:59AM,247050,10,"Methapharm, Inc.",1,,
7,Sep 22 2022 10:58AM,247049,10,"Methapharm, Inc.",1,,
8,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.,5,,
9,Sep 22 2022 10:56AM,247046,10,"Methapharm, Inc.",8,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 11:18AM,247055,19,ACG North America LLC,1,,
1,Sep 22 2022 11:15AM,247053,10,ISDIN Corporation,11,,
2,Sep 22 2022 11:09AM,247054,19,"NAPP Technologies, LLC",1,,
3,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc.",35,,
4,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 11:18AM,247055,19,ACG North America LLC,1.0,NaN,NaN
1,Sep 22 2022 11:15AM,247053,10,ISDIN Corporation,11.0,NaN,NaN
2,Sep 22 2022 11:09AM,247054,19,"NAPP Technologies, LLC",1.0,NaN,NaN
3,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc.",35.0,NaN,NaN
4,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 11:18AM,247055,19,ACG North America LLC,1.0,0.0,0.0
1,Sep 22 2022 11:15AM,247053,10,ISDIN Corporation,11.0,0.0,0.0
2,Sep 22 2022 11:09AM,247054,19,"NAPP Technologies, LLC",1.0,0.0,0.0
3,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc.",35.0,0.0,0.0
4,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3705413,79.0,5.0,5.0
12,494081,2.0,0.0,0.0
15,494060,15.0,27.0,0.0
16,247042,4.0,0.0,0.0
18,247044,1.0,0.0,0.0
19,1235034,9.0,2.0,0.0
20,988161,43.0,0.0,0.0
21,1234709,2.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3705413,79.0,5.0,5.0
1,12,494081,2.0,0.0,0.0
2,15,494060,15.0,27.0,0.0
3,16,247042,4.0,0.0,0.0
4,18,247044,1.0,0.0,0.0
5,19,1235034,9.0,2.0,0.0
6,20,988161,43.0,0.0,0.0
7,21,1234709,2.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,79.0,5.0,5.0
1,12,2.0,0.0,0.0
2,15,15.0,27.0,0.0
3,16,4.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,9.0,2.0,0.0
6,20,43.0,0.0,0.0
7,21,2.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,79.0
1,12,Released,2.0
2,15,Released,15.0
3,16,Released,4.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,5.0,0.0,27.0,0.0,0.0,2.0,0.0,3.0
Released,79.0,2.0,15.0,4.0,1.0,9.0,43.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,27.0,0.0,0.0,2.0,0.0,3.0
2,Released,79.0,2.0,15.0,4.0,1.0,9.0,43.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,27.0,0.0,0.0,2.0,0.0,3.0
2,Released,79.0,2.0,15.0,4.0,1.0,9.0,43.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()